In [ ]:
import json
import requests

# ================= 配置区域 =================
# 1. 指向本地 Ollama 接口
OLLAMA_BASE_URL = "http://192.168.9.179:11434/api/generate"
OLLAMA_MODEL_NAME = "qwen3:latest"  # 确保你的Ollama中存在这个模型



# ===========================================
INPUT_FILE = "math_problems_100.json";
# ==============================================

def distill_knowledge_local(question):
    # 针对 Qwen 优化的 Prompt，它对中文指令跟随很好
    prompt = f"""
    你是一位逻辑严密的老师。请针对以下问题，生成用于教学的思维链（Chain of Thought）。
    
    问题：{question}
    
    请务必严格按照以下JSON格式直接返回，不要包含Markdown代码块标记（如 ```json ）：
    {{
        "instruction": "{question}",
        "rationale": "这里写下详细的、分步骤的推理过程，先分析再计算，逻辑要清晰。",
        "output": "最终的简短答案"
    }}
    """
    
    try:
        # 构造Ollama API请求参数
        payload = {
            "model": OLLAMA_MODEL_NAME,
            "prompt": prompt,
            "stream": False,
            "format": "json"  # 告诉Ollama返回JSON格式响应
        }
        
        # 发送POST请求到Ollama API
        response = requests.post(OLLAMA_BASE_URL, json=payload, timeout=30)
        response.raise_for_status()  # 检查HTTP错误
        
        # 解析响应内容
        content = response.json()
        
        if isinstance(content, dict) and "response" in content:
            clean_content = content["response"].replace("```json", "").replace("```", "").strip()
            try:
                data_json = json.loads(clean_content)
                return data_json
            except json.JSONDecodeError:
                print(f"❌ JSON解析失败，模型输出了非标准格式:\n{clean_content}\n")
                return None
        else:
            print(f"❌ 响应格式异常: {content}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"网络请求出错: {e}")
        return None
    except Exception as e:
        print(f"API调用出错: {e}")
        return None

# 测试运行：我们用几个数学逻辑题来测试 Qwen 的推理生成能力
# seed_questions = [
#     "如果你有3个苹果，吃掉了1个，又买来了5个，现在有几个？",
#     "一个笼子里有鸡和兔，头有10个，脚有28只，鸡兔各多少？",
#     "用Python写一个函数，判断一个数字是不是素数。"
# ]


# if not os.path.exists(INPUT_FILE):
#     print(f"找不到文件: {INPUT_FILE}");
#     exit 0;

print(f"正在读取 {INPUT_FILE} ...");



with open(INPUT_FILE, "r", encoding="utf-8") as f:
        datasets = f.read();
        all_data = json.loads(datasets)



print(f"all_data:{all_data}");

dataset = []

print(f"正在使用本地模型 {OLLAMA_MODEL_NAME} 进行蒸馏...\n")

for question  in all_data["questions"]:
    print(f"正在处理: {question} . ..")
    content = question.get('content')  # 如果不存在'content'键，返回None
    if content is not None: 
        print(content)
        output_content = distill_knowledge_local(content)
        
        if output_content:
            # 清洗数据：有时候模型会忍不住加 ```json ... ```，我们需要去掉
            try:
                data_json = output_content
                dataset.append(data_json)
                print("✅ 生成成功！")
                print(f"  --> 思维过程片段: {data_json['rationale'][:50]}. ..")
            except Exception as e:
                print(f"❌ 数据处理异常: {e}")
        else:
            print("❌ 生成失败，无返回内容。")

# 保存结果
if dataset:
    filename = "local_distill_data.jsonl"
    with open(filename, "w", encoding="utf-8") as f:
        for entry in dataset:
            final_entry = {
                "instruction": entry["instruction"],
                "input": "",
                "output": f"【思考过程】\n{entry['rationale']}\n\n【最终答案】\n{entry['output']}"
            }
            json.dump(final_entry, f, ensure_ascii=False)
            f.write("\n")
    print(f"\n🎉 任务完成！数据已保存为 {filename}，共 {len(dataset)} 条。")


正在读取 math_problems_100.json ...
all_data:{'questions': [{'grade': '六年级', 'content': '已知一个三角形的底边长为10厘米，底边角为60度。求三角形的面积。'}, {'grade': '六年级', 'content': '计算：一个长方形的面积是 36 平方厘米，长是 8 厘米，宽是 4 厘米，求长方形的面积，并计算长方形的周长。计算方法是：面积 = 长 × 宽，周长 = 2 × (长 + 宽)。'}, {'grade': '六年级', 'content': '已知一个正方形的边长为10厘米，面积是20平方厘米。如果这个正方形的边长增加到12厘米，则新的面积是多少平方厘米？'}, {'grade': '六年级', 'content': '计算：如果一个学生在六年级时，成绩是85分，并且在三年级时是70分，那么六年级学生在三年级时成绩的平均分是多少？'}, {'grade': '六年级', 'content': '计算：一个学生在六年级数学学习一年，一共学习了 20 个课时。如果学生平均每年学习 8 个课时，那么学生一年学习多少课时？'}, {'grade': '六年级', 'content': '计算： 6年级学生完成 10 次加法题，总和为 60。  请写出答案。'}, {'grade': '六年级', 'content': '在下列题目中，选择正确的运算方法，并写出计算结果。'}, {'grade': '六年级', 'content': '计算：一个长方形的面积是 36 平方厘米，长是 8 厘米，宽是 4 厘米。求长方形的面积是多少？'}, {'grade': '六年级', 'content': '计算：一个长方形的面积是 36 平方厘米，它的周长是 24 厘米。求长方形的周长是多少？'}, {'grade': '六年级', 'content': '已知一个三角形的底边长为10厘米，底边是三角形的正方形部分，高是5厘米。求这个三角形的面积。'}, {'grade': '六年级', 'content': '请简要描述六年级学生的数学水平，例如：基础、中级、高级。'}, {'grade': '六年级', 'content': '在下面的题目中，你需要计算出六年级的学生需要完成的计算量，并